In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import time

from scripts.ImageDataset import ImageDataset
from models.FoodCNN import FoodCNN
from utils.LogClass import LogClass

def get_accuracy(prediction, gt_labels, percentage=True):
    correct = 0
    for i in range(len(prediction)):
        if percentage:
            label_prediction = np.argmax(prediction[i])
        else:
            label_prediction = prediction[i]
        if label_prediction == gt_labels[i]:
            correct += 1
    return correct / len(prediction)

# Configurazione del dataset e dei DataLoader (come nel tuo codice)
train_gt_path = './ground_truth/train_small.csv'
test_gt_path = './ground_truth/new_val_info.csv'

train_image_path = './train_set/'
test_image_path = './val_set/'

train_df = pd.read_csv(train_gt_path, header=None, names=['image_id', 'label'])
test_df = pd.read_csv(test_gt_path, header=None, names=['image_id', 'label'])

train_dataset = ImageDataset(train_df, train_image_path, train=True)
test_dataset = ImageDataset(test_df, test_image_path, train=False)
lc = LogClass('technical_report.txt')
model_name = 'resnet50'
gt_labels = test_dataset.get_all_labels()

### Train models

In [13]:
model = FoodCNN(model_name=model_name, num_classes=251, device='cuda')
start_train = time.time()
train_loss, val_loss, train_accuracies, val_accuracies = model.train_model(train_dataset, validation=0.1, num_epochs=20, lr=0.001)
end_train = time.time()

print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
lc.write(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
print(f"Training time: {end_train - start_train:.2f} s")
lc.write(f"Training time: {end_train - start_train:.2f} s")
lc.close()

/home/massimo/Documents/visual_information_and_processing/visualFoodProject/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/massimo/Documents/visual_information_and_processing/visualFoodProject/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/20: 100%|█████████████████████| 142/142 [01:29<00:00,  1.58it/s, accuracy=0.0206, loss=4.71]


Train Accuracy: 0.0206
Validation Loss: 4.9942, Validation Accuracy: 0.0558


Epoch 2/20: 100%|███████████████████████| 142/142 [01:37<00:00,  1.45it/s, accuracy=0.12, loss=5.27]


Train Accuracy: 0.1197
Validation Loss: 4.8134, Validation Accuracy: 0.1016


Epoch 3/20: 100%|████████████████████████| 142/142 [01:35<00:00,  1.48it/s, accuracy=0.2, loss=3.69]


...

Train Accuracy: 0.5757
Validation Loss: 3.8917, Validation Accuracy: 0.2311


Epoch 19/20: 100%|█████████████████████| 142/142 [01:40<00:00,  1.42it/s, accuracy=0.582, loss=3.02]


Train Accuracy: 0.5823
Validation Loss: 3.9012, Validation Accuracy: 0.2251


Epoch 20/20: 100%|█████████████████████| 142/142 [01:39<00:00,  1.42it/s, accuracy=0.582, loss=2.64]


Train Accuracy: 0.5819
Validation Loss: 3.8946, Validation Accuracy: 0.2271
Max Memory Allocated: 499.55 MB
Training time: 2238.57 s


In [3]:
lc = LogClass('technical_report.txt')
# Plot delle loss e delle accuratezze
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.savefig(f'graphs/{model_name}_loss_plot.png')
plt.clf()
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig(f'graphs/{model_name}_accuracy_plot.png')


NameError: name 'train_loss' is not defined

In [15]:
start_predict = time.time()
prediction = model.predict(test_dataset)
end_predict = time.time()

print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
lc.write(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
print(f"Prediction time: {end_predict - start_predict:.2f} s")
lc.write(f"Prediction time: {end_predict - start_predict:.2f} s")

accuracy = get_accuracy(prediction, gt_labels)
print(f'Accuracy: {accuracy}')
lc.write(f'Accuracy: {accuracy}')
lc.write('----------------------------------------------------------------------------------------------------------\n')
lc.close()

100%|██████████| 373/373 [03:06<00:00,  2.00it/s]

Max Memory Allocated: 499.55 MB
Prediction time: 186.45 s
Accuracy: 0.25967268149391526


### Simple CNN

In [2]:
from utils.LogClass import LogClass
from models.SimpleCNN import SimpleCNN
lc = LogClass('technical_report.txt')
sn = FoodCNN(model_name='simpleCNN', num_classes=251, device='cuda')
start_train = time.time()
train_loss, val_loss, train_accuracies, val_accuracies = sn.train_model(train_dataset, validation=0.1, num_epochs=20, lr=0.001)
end_train = time.time()

print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
lc.write(f"Max Memory Allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
print(f"Training time: {end_train - start_train:.2f} s")
lc.write(f"Training time: {end_train - start_train:.2f} s")
lc.close()

Epoch 1/20: 100%|████████████████████| 142/142 [01:24<00:00,  1.69it/s, accuracy=0.00288, loss=5.53]


Train Accuracy: 0.0029
Validation Loss: 5.5274, Validation Accuracy: 0.0000


Epoch 2/20: 100%|████████████████████| 142/142 [01:24<00:00,  1.68it/s, accuracy=0.00243, loss=5.52]


Train Accuracy: 0.0024
Validation Loss: 5.5300, Validation Accuracy: 0.0000


Epoch 3/20: 100%|████████████████████| 142/142 [01:26<00:00,  1.64it/s, accuracy=0.00354, loss=5.52]


...

Train Accuracy: 0.0044
Validation Loss: 5.5422, Validation Accuracy: 0.0000


Epoch 19/20: 100%|███████████████████| 142/142 [01:30<00:00,  1.56it/s, accuracy=0.00443, loss=5.52]


Train Accuracy: 0.0044
Validation Loss: 5.5422, Validation Accuracy: 0.0000


Epoch 20/20: 100%|███████████████████| 142/142 [01:34<00:00,  1.50it/s, accuracy=0.00443, loss=5.51]


Train Accuracy: 0.0044
Validation Loss: 5.5422, Validation Accuracy: 0.0000
Max Memory Allocated: 1576.48 MB
Training time: 1876.51 s
